## Leaky binary models

#### Requirements
* Neuron values are either 0 or 1 (binary). The inputs and outputs for sure, and internally as much as possible.
* Weights are real values, which are maybe bounded. 

#### Modified Heaviside

The Heaviside function is defined as 
$$
H(x):=\begin{cases}1 \quad x\geq 0\\0 \quad x< 0.\end{cases}
$$
Mathematically, it has vanishing differential, because $H$ is essentially constant. 

<!-- We will use the following leaky variant of $H$, which depends on a non-negative constant $Q$:
$$
H(x, Q):= \begin{cases} 1 \quad x\geq 0\\ Q \quad x< 0.\end{cases}
$$ -->
For our purpose, it will be convenient to use the following differential for $H$, 
$$
H_{\text{backward}} := \begin{cases} 1 \quad x\geq 0\\ -1 \quad x< 0.\end{cases}
$$


#### Internal binary representation

In order to use classical float based layers for binary computations, we will use a leaky variant of a binary tensor. Instead of $[1, 0, 0, 1]$ we will use $[1, Q, Q, 1]$, where $Q$ is a non-negative constant, which is typically small. This will help to overcome a stop of the learning process, when too many neurons are inert, that is, do not propagate any signals.

The inputs and outputs will be binary. The function `binary_in` will turn all zeros into $Q$. The differential of `binary_in` is non-trivial, it is $=1$ on a $1$, and $=-1$ on a $0$. As an example:
$$
\text{binary{\_}in}_{\text{backward at} [1, 0, 0, 1]}([y_0, y_1, y_2, y_3]) = [y_0, -y_1, -y_2, y_3]. 
$$

#### Wrapping a float layer or model

Let $L$ be a classical layer, model, or autograd function. We can wrap transform it to $L\rightarrow L_{b}$ by composing with $H$ and $\text{binary{\_}in}$:
$$
L_{b} := H \circ L \circ \text{binary{\_}in}. 
$$
This is not very intuitive, need an example here.

### Linear model overview (old) 

Old approach, included for reference

#### Assumptions
* Fixed bias to fix ideas (subject to change).

#### Architecture
Let $H(x)$ be the binary step function 
$$
H(x):=\begin{cases}1 \quad x\geq 0\\0 \quad x< 0.\end{cases}
$$

The *forward pass* of one layer reads 
$$
L_i(x,w) := H\left( \sum_{j}  w[i,j]\cdot x[j] - B \right),
$$
where $x$ are the inputs, or neuron values, and $w$ are the weights, that is the synaptic values. We denote by $B$ the bias, it is a positive constant.

From inputs to final outputs with $n$ layers, the forward pass looks like 
$$
x_0 \xrightarrow{w_0} x_1 \xrightarrow{w_1} x_2 \rightarrow \dots \rightarrow x_{n-1} \xrightarrow{w_{n-1}} x_n,
$$
with 
$$
x_i = L(x_{i-1}, w_{i-1}), \quad x_0 = \text{given inputs}.
$$
Again, $x$ has only entries in $\{0, 1\}$, and $w$ has floats as entries (bounded?).  

The *backward pass* is given by
$$
gx_n \xrightarrow{gw_{n-1}} gx_{n-1} \xrightarrow{gw_{n-2}} gx_{n-2} \rightarrow \dots \rightarrow gx_{1} \xrightarrow{gw_{0}} gx_0,
$$
where 
\begin{split}
gx_i &= G(gx_{i+1}, x_{i+1}, x_{i}, w_{i})\\
gw_i &= W(gx_{i+1}, x_{i+1}, x_i) \\
gx_n &= Neq(\text{desired outputs}, x_n).
\end{split}
We will define $G$, $W$, and $Neq$ in the following. 

As for the forward pass, $gx$ has non-negative floats as entries, and $gw$ has floats as entries. Moreover, the shape of $x_i$ equals the shape of $gx_i$, and the shape of $w_i$ equals the shape of $gw_i$.

#### Backward pass formulas

##### Neq
For $x, y \in \{0, 1\}$, we set
$$
Neq(x, y) = \begin{cases} 0 \quad  &x=y, \\ 1 \quad &x\neq y. \end{cases} 
$$
This is generalized to $x$ and $y$ of the same shape by applying entry by entry. The signature is
```
Neq: (bool, shape) x (bool, shape) -> (bool, shape)  
```

##### W
Let $n$ be the shape of $x$ and $gx$, and let $m$ be the shape of $x'$. We define $W(gx, x, x')$, which is of shape $n\times m$, by 
$$
W(gx, x, x') := \left( -gx[i] \cdot P(x[i], x'[j]) \right)_{i,j},
$$
where 
$$
P(0, 1) = 1, \quad P(1, 1) = -1, \quad P(0, 0) = Q, \quad P(1, 0) = -Q,  
$$
with a non-negative $Q<1$. The value of $Q$ is connected to the learning rate (reference???). It is similar to a negative slope for a leaky ReLU. 

In order to compute $P$, we can use 
$$
P(x,y) =  (1 - 2 \cdot x) \cdot (Q  + (1 - Q) \cdot y) 
$$ 

The signature is
```
W: (bool, n) x (bool, n) x (bool, m) -> ({-1, -Q, 0, Q, 1}, n x m).  
```

##### G
Let $n$ be the shape of $x$ and $gx$, and $m$ the shape of $x'$, then $w'$ must have shape $n\times m$. We define 
$$
G(gx, x, x', w') = \left(ReLU\left( \sum_{i} gx[i]\cdot Seq(x[i], x'[j])  \cdot w[i,j] \right) \right)_{j},
$$
where 
$$
Seq(x,y) = (2x-1)\cdot (2y-1)  = \begin{cases} 1 \quad  &x=y, \\ -1 \quad &x\neq y. \end{cases}
$$
The signature is 
```
G: (bool, n) x (bool, n) x (bool, m) x (float, n x m) -> (non-negative, m).
```

#### Unsupervised methods

##### Oja's rule
(This doesn't seem to work well)

Setup: $x \xrightarrow{w} y$, we don't have a gradient for $y$ or $x$, but one for $w$:
$$
gw[i,j] := y[i]\cdot (y[i]\cdot w[i,j] - x[j]\cdot B). 
$$
(Or some other positive multiple of $B$.)

We have only non-zero gradients if $y[i]=1$. Additionally, we get a *positive* gradient if 
* $x[j]=1$ and $w[i,j]>B$, 
* or $x[j]=0$ and $w[i,j]>0$. 

We get a *negative* gradient if 
* $x[j]=1$ and $w[i,j]<B$, 
* or $x[j]=0$ and $w[i,j]<0$.

This implies two behaviours:
* If $x[j]$ is inert then the weight will converge to zero.
* The weight will be bounded by $B$ or some other positive multiple of $B$.

##### Generalized Hebbian algorithm

Setup: $x \xrightarrow{w} y$, we don't have a gradient for $y$ or $x$, but one for $w$:
$$
gw[i,j] := y[i]\cdot (- x[j]\cdot B + \sum_{k} y[k]\cdot w[k,j]). 
$$
(Or some other to be determined positive multiple of $B$.)

### Pytorch implementation

In [1]:
import torch

#### Heaviside and ReLU

In [2]:
input = torch.tensor([-1, 0, 1], dtype=torch.float32)
value = torch.tensor([1], dtype=torch.float32)

def H_forward(input):
    return torch.heaviside(input, value)

class HeavisideBN(torch.autograd.Function):
    @staticmethod
    # B and Q are added as tensors
    def forward(ctx, input):
        output = H_forward(input)
        #print("In forward: {}".format(output))
        ctx.save_for_backward(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        output, = ctx.saved_tensors

        grad_input  = None
        grad_input = grad_output * (2. * output - 1.)
    
        return grad_input

In [3]:
H = HeavisideBN.apply

In [4]:
H_forward(input)==H(input)

input.requires_grad = True

H(input).backward(torch.tensor([1., 1., 1.]))
input.grad

tensor([-1.,  1.,  1.])

#### binary_in

In [5]:
class BinaryIn(torch.autograd.Function):
    @staticmethod
    # B and Q are added as tensors
    def forward(ctx, input, Q):
        output = input + Q * (1.-input)
        #print("In forward: {}".format(output))
        ctx.save_for_backward(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors

        grad_input  = None
        grad_input = grad_output * (2. * input - 1.)
    
        return grad_input, None

In [6]:
binary_in = BinaryIn.apply

In [7]:
input = torch.tensor([1, 0, 0, 1], dtype=torch.float32, requires_grad=True)

t = binary_in(input, torch.tensor([0.01]))
print(t)

t.backward(torch.tensor([1., 1., 1., 1.]))
input.grad

tensor([1.0000, 0.0100, 0.0100, 1.0000], grad_fn=<BinaryInBackward>)


tensor([ 1., -1., -1.,  1.])

#### Forward pass

In [8]:
weights = torch.tensor([[0.2, 0.1, 1.1], [0, -0.4, 0.3]], dtype=torch.float32, requires_grad=True)
x = torch.tensor([[0, 1, 1]], dtype=torch.float32)
weights, x

(tensor([[ 0.2000,  0.1000,  1.1000],
         [ 0.0000, -0.4000,  0.3000]], requires_grad=True),
 tensor([[0., 1., 1.]]))

In [9]:
torch.matmul(x, weights.t())

tensor([[ 1.2000, -0.1000]], grad_fn=<MmBackward0>)

In [10]:
def forward_bool(weights, input, B):
    return H(torch.matmul(input, weights.t()) - B)

In [11]:
forward_bool(weights, x, torch.tensor([1.]))

tensor([[1., 0.]], grad_fn=<HeavisideBNBackward>)

#### Neq

In [12]:
t = torch.tensor([[0, 1, 1, 1]], dtype=torch.float32)
s = torch.tensor([[0, 0, 0, 1]], dtype=torch.float32)
s, t 

(tensor([[0., 0., 0., 1.]]), tensor([[0., 1., 1., 1.]]))

In [13]:
def neq(s,t):
    return 1-(s==t).to(torch.float32)

In [14]:
neq(s,t)

tensor([[0., 1., 1., 0.]])

#### W

In [15]:
x = torch.tensor([[0,1]], dtype=torch.float32)
y = torch.tensor([[1,1,0]], dtype=torch.float32, requires_grad=True)
x, y

(tensor([[0., 1.]]), tensor([[1., 1., 0.]], requires_grad=True))

In [16]:
Q = torch.tensor([0.8], dtype=torch.float32)

first_factor = 1 - 2*x
second_factor = Q + (1-Q) * y
first_factor, second_factor

(tensor([[ 1., -1.]]),
 tensor([[1.0000, 1.0000, 0.8000]], grad_fn=<AddBackward0>))

In [17]:
first_factor.shape

torch.Size([1, 2])

In [18]:
torch.matmul(torch.reshape(first_factor, first_factor.shape + (1,)),  
         torch.reshape(second_factor, (second_factor.shape[0], 1) + second_factor.shape[1:]))

tensor([[[ 1.0000,  1.0000,  0.8000],
         [-1.0000, -1.0000, -0.8000]]], grad_fn=<UnsafeViewBackward0>)

In [19]:
# bring in gx
gx = torch.tensor([[1,0]], dtype=torch.float32)
first_factor = gx * (1 - 2*x)

In [20]:
torch.matmul(torch.reshape(first_factor, first_factor.shape + (1,)),  
         torch.reshape(second_factor, (second_factor.shape[0], 1) + second_factor.shape[1:]))

tensor([[[1.0000, 1.0000, 0.8000],
         [0.0000, 0.0000, 0.0000]]], grad_fn=<UnsafeViewBackward0>)

In [21]:
def W(gx, x, y, q):
    first_factor = (-1) * gx * (1 - 2*x)
    second_factor = q + (1-q) * y
    return torch.matmul(torch.reshape(first_factor, first_factor.shape + (1,)),  
                        torch.reshape(second_factor, (second_factor.shape[0], 1) + second_factor.shape[1:]))

In [22]:
W(gx,x,y,Q)

tensor([[[-1.0000, -1.0000, -0.8000],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<UnsafeViewBackward0>)

#### G

In [23]:
x = torch.tensor([[0,1]], dtype=torch.float32)
y = torch.tensor([[1,1,0]], dtype=torch.float32, requires_grad=True)

seq_first_factor = 2*x - 1
seq_second_factor = 2*y - 1
seq_first_factor, seq_second_factor

(tensor([[-1.,  1.]]), tensor([[ 1.,  1., -1.]], grad_fn=<SubBackward0>))

In [24]:
def seq(x, y):
    seq_first_factor = 2*x - 1
    seq_second_factor = 2*y - 1
    return torch.matmul(torch.reshape(seq_first_factor, seq_first_factor.shape +(1,)),  
             torch.reshape(seq_second_factor, (seq_second_factor.shape[0], 1) + seq_second_factor.shape[1:]))

In [25]:
seq(x, y)

tensor([[[-1., -1.,  1.],
         [ 1.,  1., -1.]]], grad_fn=<UnsafeViewBackward0>)

In [26]:
ReLU = torch.nn.ReLU()

def G(gx, x, y, w):
    new_w = seq(x, y) * torch.reshape(w, (1,) + w.shape) 
    linear = torch.matmul(torch.reshape(gx, (gx.shape[0], 1) + gx.shape[1:]), new_w)
    return ReLU(torch.reshape(linear, (linear.shape[0],) + linear.shape[2:]))

In [27]:
B = torch.tensor([0.5])
print("gx : {} \n x: {} \n y: {}\n w: {} \n B: {}".format(gx, x, y, weights, B))
G(gx, x, y, weights)


gx : tensor([[1., 0.]]) 
 x: tensor([[0., 1.]]) 
 y: tensor([[1., 1., 0.]], requires_grad=True)
 w: tensor([[ 0.2000,  0.1000,  1.1000],
        [ 0.0000, -0.4000,  0.3000]], requires_grad=True) 
 B: tensor([0.5000])


tensor([[0.0000, 0.0000, 1.1000]], grad_fn=<ReluBackward0>)

#### Oja's rule

In [28]:
# Warning: y are the inputs, x the outputs

def oja(x, y, w, B, epsilon=torch.tensor(0.001)):
    x_reshape = torch.reshape(x, x.shape+(1,))
    first_matrix = x_reshape * x_reshape * w
    second_matrix =  (1 + epsilon) * B* torch.matmul(x_reshape, torch.reshape(y, (y.shape[0], 1) + y.shape[1:]))
    return first_matrix-second_matrix

In [29]:
w_oja = torch.tensor([[1.0276, 0.3],
        [0.45, 1.1325]], requires_grad=True)
x_oja = torch.tensor([[0., 1.]])
y_oja = torch.tensor([[0., 1.]])
B = torch.tensor([1.])

oja(x_oja, y_oja, w_oja, B)

tensor([[[0.0000, 0.0000],
         [0.4500, 0.1315]]], grad_fn=<SubBackward0>)

#### Generalized Hebbian algorithm

In [30]:
# Warning: y are the inputs, x the outputs

def hebbian(x, y, w, B, epsilon=torch.tensor(0.5)):
    x_reshape = torch.reshape(x, x.shape+(1,))
    vector = torch.matmul(x, w)
    first_matrix = torch.matmul(x_reshape, torch.reshape(vector, (vector.shape[0], 1) + vector.shape[1:]))   
    second_matrix =  (1 + epsilon) * B* torch.matmul(x_reshape, torch.reshape(y, (y.shape[0], 1) + y.shape[1:]))
    return first_matrix-second_matrix

In [31]:
w_heb = torch.tensor([[1.0276, 0.3],
        [0.45, 1.1325]], requires_grad=True)
x_heb = torch.tensor([[0., 1.]])
y_heb = torch.tensor([[0., 1.]])
B = torch.tensor([1.])

hebbian(x_heb, y_heb, w_heb, B)

tensor([[[ 0.0000,  0.0000],
         [ 0.4500, -0.3675]]], grad_fn=<SubBackward0>)

#### Autograd function

In [32]:
class LinearBool(torch.autograd.Function):
    @staticmethod
    # B and Q are added as tensors
    def forward(ctx, input, weight, b, q, oja_lr, heb_lr):
        output = forward_bool(weight, input, b)
        #print("In forward: {}".format(output))
        ctx.save_for_backward(input, weight, b, q, oja_lr, heb_lr, output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, weight, b, q, oja_lr, heb_lr, output = ctx.saved_tensors

        grad_input = grad_weight = grad_b = grad_q = grad_oja_lr = grad_heb_lr = None

        grad_input = G(grad_output, output, input, weight)
        grad_weight = W(grad_output, output, input, q)

        #additional "learning", independent of gradients
        if oja_lr!=0.:
            grad_weight += oja_lr *  oja(output, input, weight, b)  
            #print(oja(output, input, weight, b))

        if heb_lr!=0.:
            grad_weight += heb_lr *  hebbian(output, input, weight, b)  
    
        return grad_input, grad_weight, grad_b, grad_q, grad_oja_lr, grad_heb_lr
        

##### Tests

In [33]:
# Forward

linear = LinearBool.apply
#print(y, weights, B, Q)
oja_lr = torch.tensor([0.])

output = linear(y, weights, B, Q, torch.tensor([0.1]), torch.tensor([0.]))
output

tensor([[0., 0.]], grad_fn=<LinearBoolBackward>)

In [34]:
# Backward
desired_output = torch.tensor([[1, 1]], dtype=torch.float32)
gradient = neq(output, desired_output)

output.backward(gradient)

In [35]:
weights.grad, y.grad

(tensor([[-1.0000, -1.0000, -0.8000],
         [-1.0000, -1.0000, -0.8000]]),
 tensor([[0.0000, 0.3000, 1.4000]]))

#### Comparison

In order to compare performance with a non binary approach, we introduce a simple linear layer with constant bias and ReLU.  

In [36]:
def linear_compare(y, weights, B):
    l = torch.matmul(y, weights.t()) - B
    return torch.nn.ReLU()(l)

def loss_gradient(o, output_bn):
    return output_bn-o

#### Weight initialization

Start with very simple approach of random weights.

In [37]:
def weights_init(shape, scale, B):

    first = scale * torch.rand(shape, dtype=torch.float32, requires_grad=False)

    # make sure the rows sum up to B
    first += B * torch.tensor([1/shape[1]], dtype=torch.float32) 

    first.requires_grad = True
    return first


#### Models helper

In [38]:
class LinearBN(torch.nn.Module):

    def __init__(self, in_features, out_features, Q=0.05, B=1., oja_lr=0., heb_lr = 0., weights=None):
        super(LinearBN, self).__init__()

        self.Q = torch.tensor([Q], dtype=torch.float32)
        self.B = torch.tensor([B], dtype=torch.float32)
        self.oja_lr = torch.tensor([oja_lr], dtype=torch.float32)
        self.heb_lr = torch.tensor([heb_lr], dtype=torch.float32)

        # TODO correct scale and initialization?
        scale = 2*B
        
        if weights is None:
            self.weights = torch.nn.Parameter(weights_init((out_features, in_features), scale, self.B))
        else: 
            self.weights = torch.nn.Parameter(weights)

    def forward(self, input):
        # transform to batch
        if len(input.shape) == 1:
            input = torch.reshape(input, (1,) + input.shape)
        return LinearBool.apply(input, self.weights, self.B, self.Q, self.oja_lr, self.heb_lr)
        


In [39]:
t = LinearBN(2, 3)

In [40]:
t.weights

Parameter containing:
tensor([[2.4116, 2.4650],
        [0.5220, 1.6091],
        [0.8054, 1.1164]], requires_grad=True)

In [41]:
batch_size = 8
batch = weights_init((batch_size, 2), 1., B)
batch.shape

torch.Size([8, 2])

In [42]:
out = t(batch)
out

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], grad_fn=<LinearBoolBackward>)

In [43]:
optimizer = torch.optim.SGD(t.parameters(), lr=0.1)

In [44]:
optimizer.zero_grad()

In [45]:
desired = torch.tensor([[1., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 0.],
        [1., 0., 0.]])

In [46]:
with torch.no_grad():
    grad = neq(out, desired)

grad

tensor([[0., 1., 0.],
        [0., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.],
        [0., 0., 0.],
        [1., 0., 0.],
        [1., 1., 1.],
        [0., 1., 1.]])

In [47]:
out.backward(grad)

In [48]:
t.weights.grad

tensor([[1.7984, 2.0343],
        [5.4038, 4.2580],
        [4.8560, 4.4350]])

In [49]:
optimizer.step()

In [50]:
t.weights

Parameter containing:
tensor([[ 2.2318,  2.2616],
        [-0.0183,  1.1833],
        [ 0.3198,  0.6729]], requires_grad=True)

#### Learning tools

##### Supervised

In [51]:
import numpy as np

def learn(inputs, desired_outputs, model, weights, bias, learning_rate, 
            binarizatio_fn=lambda input: torch.heaviside(input, torch.tensor([0.])),
            loss_gradient = neq, print_it=False):

    for w in weights:
        w.grad = None

    for b in bias:
        b.grad = None

    eq_counter = 0
    for i, o in zip(inputs, desired_outputs):
        output = model(i)
        output_bn = binarizatio_fn(output)
        if print_it:
            with torch.no_grad():
                print("{} -> {}".format(i.numpy(), output_bn.numpy()))

        if torch.all(o==output_bn):
            eq_counter += 1
            
        gradient = loss_gradient(o, output_bn)
            
        output.backward(gradient)
        with torch.no_grad():
            for w in weights:
                if print_it:
                    print("Weight grads: {}".format(w.grad))
                w -= learning_rate * w.grad
                w.grad = None
            for b in bias:
                b -= learning_rate * b.grad
                b.grad = None
                    
    if eq_counter == len(inputs):
        return "Done"

In [52]:
def learn_model(inputs, desired_outputs, model, learning_rate, 
            binarizatio_fn=lambda input: torch.heaviside(input, torch.tensor([0.])),
            loss_gradient = neq, print_it=False):

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    eq_counter = 0
    for i, o in zip(inputs, desired_outputs):
        optimizer.zero_grad()

        output = model(i)
        output_bn = binarizatio_fn(output)
        
        if print_it:
            with torch.no_grad():
                print("{} -> {}".format(i.numpy(), output_bn.numpy()))

        if torch.all(o==output_bn):
            eq_counter += 1
        
        gradient = loss_gradient(o, output_bn)
            
        output.backward(gradient)
        if print_it:
            for w in model.parameters():
                print("Weight grads: {}".format(w.grad))
        optimizer.step()
                    
    if eq_counter == len(inputs):
        return "Done"

##### Unsupervised

In [53]:
def learn_unsupervised(inputs, model, learning_rate, print_it=False):

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for i in inputs:
        optimizer.zero_grad()

        output = model(i)
        
        if print_it:
            with torch.no_grad():
                print("{} -> {}".format(i.numpy(), output.numpy()))
        
        gradient = torch.zeros(output.shape, dtype=output.dtype)
            
        output.backward(gradient)
        if print_it:
            for w in model.parameters():
                print("Weight grads: {}".format(w.grad))
        optimizer.step()

### Examples

All examples start with random initialisation of weights in the range $(0, 1)$.

#### Copy one neuron

Architecture: `input_neuron -> output_neuron` 

Desired behavior: `0->0, 1->1`

Explicitly, for forward $y=H(x\cdot w-B)$. If $x=0$ then $y=0$, which is desired behavior. Suppose $x=1$ and $y=0$. Desired value is $y'=1$. For backward: the first gradient is $gy= Eq(y', y)=1$. The weight gradient reads $gw = W(gy, y, x)= 1$, pointing to increasing $w$, which is correct behavior. 

Once correct behavior is established, the weight gradients are zero. 

In [54]:
y = torch.tensor([1.], requires_grad=True)
desired_output = torch.tensor([1.], requires_grad=True)
# start with negative weight, it will become positive after learning 
w = torch.tensor([-0.6], requires_grad=True)
print("Starting with weight {}".format(w))
gradient = None
limit = 5
counter = 0

while  counter<limit:
    output = linear(y, w, B, Q, torch.tensor([0.02]), torch.tensor([0.]))
    print("Output: {}".format(output))

    gradient = neq(output, desired_output)
    output.backward(gradient)
    print("Weight gradient: {}".format(w.grad))

    with torch.no_grad():
        w -= w.grad
    print("Changing weight to {}".format(w))
    w.grad = None

    counter += 1

Starting with weight tensor([-0.6000], requires_grad=True)
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([-1.])
Changing weight to tensor([0.4000], requires_grad=True)
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([-1.])
Changing weight to tensor([1.4000], requires_grad=True)
Output: tensor([1.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([0.0080])
Changing weight to tensor([1.3920], requires_grad=True)
Output: tensor([1.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([0.0078])
Changing weight to tensor([1.3842], requires_grad=True)
Output: tensor([1.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([0.0077])
Changing weight to tensor([1.3765], requires_grad=True)


#### Copy one neuron with a neuron in between

Architecture: `input_neuron -> other_neuron -> output_neuron` 

Desired behavior: `0->0, 1->1`

The backpropagation of 
$$
x_\text{in} \xrightarrow{w_\text{in}} x_\text{other} \xrightarrow{w_\text{other}} x_{\text{out}}
$$
is computed as follows
$$
gx_\text{out} \xrightarrow{gw_\text{other}} gx_\text{other} \xrightarrow{gw_\text{in}} gx_\text{in}$$
Independent of the $x_{\text{other}}$ values, $gw_\text{other}$ will be positive if the desired value for $x_\text{out}$ is not reached. Once $w_\text{other}$ is large enough, $gx_\text{other}$ will become positive if $x_\text{out}=0$ is not desired and $x_{\text{other}}=0$. This will push $gw_\text{in}$ to positive, fixing $w_\text{in}$. The model will solve the task.  



In [55]:
y = torch.tensor([1.], requires_grad=True)
desired_output = torch.tensor([1.], requires_grad=True)

# start with negative weight, it will become positive after learning 
w_in = torch.tensor([-0.6], requires_grad=True)
w_other = torch.tensor([-0.9], requires_grad=True)
weights = [w_in, w_other]
print("Starting with weights {}".format(weights))

def model(input): 
    return linear(linear(input, w_in, B, Q, torch.tensor([0.]), torch.tensor([0.])), 
                                w_other, B, Q, torch.tensor([0.]), torch.tensor([0.]))

gradient = None
limit = 10
counter = 0

while gradient != torch.tensor([0.]) and counter < limit:
    output = model(y)
    print("Output: {}".format(output))

    gradient = neq(output, desired_output)
    output.backward(gradient)
    print("Weight gradients: {}".format([w.grad for w in weights]))

    with torch.no_grad():
        for w in weights:
            w -= w.grad
            w.grad = None            
    print("Changing weight to {}".format(weights))
 
    counter += 1

Starting with weights [tensor([-0.6000], requires_grad=True), tensor([-0.9000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([0.]), tensor([-0.8000])]
Changing weight to [tensor([-0.6000], requires_grad=True), tensor([-0.1000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([0.]), tensor([-0.8000])]
Changing weight to [tensor([-0.6000], requires_grad=True), tensor([0.7000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([-0.7000]), tensor([-0.8000])]
Changing weight to [tensor([0.1000], requires_grad=True), tensor([1.5000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([-1.5000]), tensor([-0.8000])]
Changing weight to [tensor([1.6000], requires_grad=True), tensor([2.3000], requires_grad=True)]
Output: tensor([1.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([0.]), tensor

#### Always disagree with one neuron

Architecture: `input_neuron -> output_neuron` 

Desired behavior: `1->0`. Note that `0->0` is automatic. 

Explicitly, for forward $y=H(x\cdot w-B)$. If $x=1$ and $y=1$, then the weight gradient is
$
gw = -1
$
pushing the weights in the negative direction, which is correct behavior. 

In [56]:
inputs = [torch.tensor([1.], requires_grad=True)]
desired_outputs = [torch.tensor([0.])]
weights = [torch.tensor([1.], requires_grad=True)]

def model(input):
    return linear(input, weights[0], B, Q, torch.tensor([0.]), torch.tensor([0.]))

for i in range(10):
    learn(inputs, desired_outputs, model, weights, [], learning_rate=0.1)

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

New weights: [tensor([0.9000], requires_grad=True)]
[1.] -> [0.]


#### Switch positions

Architecture: `input_neuron_0, input_neuron_1  -> output_neuron_0, output_neuron_1` 

Desired behavior: `I:(1,0)->(0,1), II:(0,1)->(1,0), III:(1,1)->(1,1)`

Explicitly, for forward $y_i=H(x_0\cdot w_{i,0} + x_1\cdot w_{i,1} - B)$. Let's consider the weights one by one:
* $w_{0,0}$: since `I` and `III` send contradictory impulses at first, no changes.
* $w_{0,1}$: increasing, so it will fix `II` and have a strong angle on `III`. This will allow $w_{0,0}$ to focus on `I` and fix it. 
* $w_{1,0}$: Same as $w_{0,1}$.
* $w_{1,1}$: Same as $w_{0,0}$.

In [57]:
inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True),
          torch.tensor([[1., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[0., 1.]]), torch.tensor([[1., 0.]]), torch.tensor([[1., 1.]])]

# initialization must be random
weights = [torch.tensor([[0.1, -0.2], 
                         [-0.22, -0.01]], requires_grad=True)]

def model(input):
    return linear(input, weights[0], B, Q, torch.tensor([0.01]), torch.tensor([0.]))

for i in range(50):
    done = learn(inputs, desired_outputs, model, weights, [],  learning_rate=0.1)
    if done=="Done":
        print("Done after epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Done after epoch 15
New weights: [tensor([[0.9186, 1.0154],
        [1.0007, 0.9854]], requires_grad=True)]
[[1. 0.]] -> [[0. 1.]]
[[0. 1.]] -> [[1. 0.]]
[[1. 1.]] -> [[1. 1.]]


In [58]:
# Comparison with ReLUs 

# initialization must be random
# Following weights won't work, because ReLU does not propagate grads if value = 0
# weights = [torch.tensor([[0.1, -0.2], 
#                         [-0.22, -0.01]], requires_grad=True)]
weights = [torch.tensor([[0.1, 0.13], 
                            [0.05, 0.07]], requires_grad=True)]

#bias = [torch.tensor([0.5], requires_grad=True)]
bias = []
binarization_fn = lambda input: torch.heaviside(input-0.2, torch.tensor([0.]))

def model(input):
    return linear_compare(input, weights[0], torch.tensor([0.])) 

for i in range(20):
    done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.01, 
                    binarizatio_fn=binarization_fn, loss_gradient=loss_gradient)
    #done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.1)
    if done=="Done":
        print("Done after epoch {}".format(i))
        break

print("New weights: {}".format(weights))
print("New bias: {}".format(bias))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), binarization_fn(model(input)).numpy()))

Done after epoch 12
New weights: [tensor([[0.1000, 0.2000],
        [0.2000, 0.1000]], requires_grad=True)]
New bias: []
[[1. 0.]] -> [[0. 1.]]
[[0. 1.]] -> [[1. 0.]]
[[1. 1.]] -> [[1. 1.]]


In [59]:
# with model class definition
model = LinearBN(2,2, oja_lr=0.04)

for i in range(600):
    done = learn_model(inputs, desired_outputs, model, learning_rate=0.1)
    #if done=="Done":
    #    print("Done after epoch {}".format(i))
    #    break

print("New weights: {}".format(model.weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

New weights: Parameter containing:
tensor([[0.5051, 1.0114],
        [1.0013, 0.5055]], requires_grad=True)
[[1. 0.]] -> [[0. 1.]]
[[0. 1.]] -> [[1. 0.]]
[[1. 1.]] -> [[1. 1.]]


#### Hidden representation

Architecture: `input_0, input_1  -> hidden_0, hidden_1, hidden_2 -> output` 

Desired behavior: `I:(1,0)->(1), II:(0,1)->(1), III:(1,1)->(0)`

This cannot be learned with one layer only, because `I` and `II` imply two weights that are larger than the bias. This contradicts `III`.

Questions
* Why is $Q=0.1$ working, but other values are not?


In [60]:
inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True),
          torch.tensor([[1., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1.]]), torch.tensor([[1.]]), torch.tensor([[0.]])]

use_Q = torch.tensor([0.1])

# initialization must be random
weights = [torch.tensor([[0.1, -0.2], 
                         [-0.22, -0.01],
                         [0.22, 0.01]], requires_grad=True), 
           torch.tensor([[0.1, 0.16, -0.2]], requires_grad=True)]

def model(input):
    hidden = linear(input, weights[0], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    output = linear(hidden, weights[1], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    return output

epochs = 1000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.11)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Finished in epoch 46
New weights: [tensor([[1.0068, 0.9689],
        [1.0170, 1.0247],
        [0.6346, 0.4538]], requires_grad=True), tensor([[ 0.1440,  1.0950, -0.2550]], requires_grad=True)]
[[1. 0.]] -> [[1.]]
[[0. 1.]] -> [[1.]]
[[1. 1.]] -> [[0.]]


#### Hidden representation 2

Architecture: `input_0, input_1  -> hidden_0, hidden_1 -> output` 

Desired behavior: `I:(1,0)->(1), II:(0,1)->(1), III:(1,1)->(0)`

This cannot be learned with one layer only, because `I` and `II` imply two weights that are larger than the bias. This contradicts `III`.

In [61]:
inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True),
          torch.tensor([[1., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1.]]), torch.tensor([1.]), torch.tensor([[0.]])]

use_Q = torch.tensor([0.1])

# initialization must be random
weights = [torch.tensor([[0.09, 0.19],
                            [0.19, 0.09]], requires_grad=True), 
           torch.tensor([[0.19, -0.0]], requires_grad=True)]

def model(input, B=torch.tensor([0.2])):
    hidden = linear(input, weights[0], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    output = linear(hidden, weights[1], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    return output

epochs = 3000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.01)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Finished in epoch 63
New weights: [tensor([[0.2000, 0.2015],
        [0.1900, 0.0900]], requires_grad=True), tensor([[ 0.2020, -0.0150]], requires_grad=True)]
[[1. 0.]] -> [[1.]]
[[0. 1.]] -> [[1.]]
[[1. 1.]] -> [[0.]]


In [62]:
### Comparison with ReLUs

inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True),
          torch.tensor([[1., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1.]]), torch.tensor([[1.]]), torch.tensor([[0.]])]

bias = []
binarization_fn = lambda input: torch.heaviside(input-0.2, torch.tensor([0.]))

# initialization must be random
weights = [torch.tensor([[0.1, 0.2], 
                         [0.22, 0.1]], requires_grad=True), 
           torch.tensor([[0.1, 0.16]], requires_grad=True)]

zero = torch.tensor([0.])

def model(input):
    hidden = linear_compare(input, weights[0], zero)
    output = linear_compare(hidden, weights[1], zero)
    return output

epochs = 10000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.05,
                    binarizatio_fn=binarization_fn, loss_gradient=loss_gradient)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), binarization_fn(model(input)).numpy()))

Finished in epoch 1572
New weights: [tensor([[-0.4186,  0.4077],
        [ 0.4946, -0.0263]], requires_grad=True), tensor([[0.5427, 0.4234]], requires_grad=True)]
[[1. 0.]] -> [[1.]]
[[0. 1.]] -> [[1.]]
[[1. 1.]] -> [[0.]]


#### Locked state

Architecture: `input_0, input_1  -> hidden_0, hidden_1 -> output` 

Desired behavior: `I:(1,0)->(1), II:(0,1)->(0)`

Initial weights: such that `(1,0)` and `(0,1)` are mapped to the same element in the hidden layer.   

In [63]:
inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1.]]), torch.tensor([[0.]])]

use_Q = torch.tensor([0.1])

# initialization as required by the problem
weights = [torch.tensor([[0.03, 1.98],
                            [-0.01, 0.75]], requires_grad=True), 
           torch.tensor([[0.2070, 0.32]], requires_grad=True)]

print("Initial weights: {}".format(weights))

def model(input, B=torch.tensor([1.])):
    hidden = linear(input, weights[0], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    output = linear(hidden, weights[1], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    return output

epochs = 100

for i in range(epochs):
    #print("Epoch: {}".format(i))
    #print("Start weights: {}".format(weights))
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.05)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Initial weights: [tensor([[ 0.0300,  1.9800],
        [-0.0100,  0.7500]], requires_grad=True), tensor([[0.2070, 0.3200]], requires_grad=True)]
Finished in epoch 56
New weights: [tensor([[0.9946, 2.0765],
        [1.0122, 0.8522]], requires_grad=True), tensor([[0.4870, 1.0050]], requires_grad=True)]
[[1. 0.]] -> [[1.]]
[[0. 1.]] -> [[0.]]


In [64]:
### Comparison with ReLUs

inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1.]]), torch.tensor([[0.]])]

bias = []
binarization_fn = lambda input: torch.heaviside(input-0.2, torch.tensor([0.]))

# initialization as required by the problem
weights = [torch.tensor([[0.23, 0.23],
                            [-0.01, 0.04]], requires_grad=True), 
           torch.tensor([[0.2070, 0.32]], requires_grad=True)]

zero = torch.tensor([0.])

def model(input):
    hidden = linear_compare(input, weights[0], zero)
    output = linear_compare(hidden, weights[1], zero)
    return output

epochs = 10000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.05,
                    binarizatio_fn=binarization_fn, loss_gradient=loss_gradient)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), binarization_fn(model(input)).numpy()))

Finished in epoch 15
New weights: [tensor([[ 0.4596,  0.2300],
        [-0.0100,  0.0400]], requires_grad=True), tensor([[0.4489, 0.3200]], requires_grad=True)]
[[1. 0.]] -> [[1.]]
[[0. 1.]] -> [[0.]]


#### Getting through layers

Architecture: `input_0, input_1  -> layer_1 -> layer_2 -> ... -> layer_d` 

All layers are `2`-dimensional, and the depth `d` is variable. 

Desired behavior: `(1,0)->(1,0), (0,1)->(0,1)`

Initial weights: random

The task is to get through all the randomly initialized layers.

In [66]:
inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1., 0.]]), torch.tensor([[0., 1.]])]

# constants
use_Q = torch.tensor([0.05])
b = 0.2 

# depth
d = 5

# random initialization
weights = []
for i in range(d):
    weights.append(weights_init((2, 2), 0.2, b))

print("Initial weights: {}".format(weights))

def model(input, B=torch.tensor([b])):
    all_layers = {0: input}
    for i in range(d):
        all_layers[i+1] = linear(all_layers[i], weights[i], B, use_Q, torch.tensor([0.]), torch.tensor([0.]))
    return all_layers[d]

epochs = 1000

for i in range(epochs):
    #print("Epoch: {}".format(i))
    #print("Start weights: {}".format(weights))
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.1)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Initial weights: [tensor([[0.2689, 0.2052],
        [0.2187, 0.2402]], requires_grad=True), tensor([[0.1026, 0.2902],
        [0.1476, 0.2708]], requires_grad=True), tensor([[0.2030, 0.1806],
        [0.1867, 0.2521]], requires_grad=True), tensor([[0.2205, 0.2756],
        [0.1430, 0.2161]], requires_grad=True), tensor([[0.2566, 0.1828],
        [0.1288, 0.2771]], requires_grad=True)]
New weights: [tensor([[0.2556, 0.2000],
        [0.2001, 0.2001]], requires_grad=True), tensor([[0.0188, 0.1811],
        [0.0593, 0.1420]], requires_grad=True), tensor([[-0.0022,  0.1955],
        [ 0.0731,  0.1268]], requires_grad=True), tensor([[ 0.1923,  0.0050],
        [ 0.1815, -0.0042]], requires_grad=True), tensor([[0.2216, 0.1478],
        [0.0488, 0.1021]], requires_grad=True)]
[[1. 0.]] -> [[0. 0.]]
[[0. 1.]] -> [[0. 0.]]


In [67]:
# With ReLUs

inputs = [torch.tensor([[1., 0.]], requires_grad=True), 
          torch.tensor([[0., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1., 0.]]), torch.tensor([[0., 1.]])]


# depth
d = 5

# random initialization
weights = []
for i in range(d):
    weights.append(torch.empty((2, 2)))
    torch.nn.init.kaiming_uniform_(weights[-1])
    weights[-1].requires_grad = True


print("Initial weights: {}".format(weights))

zero = torch.tensor([0.])

def model(input):
    all_layers = {0: input}
    for i in range(d):
        all_layers[i+1] = linear_compare(all_layers[i], weights[i], zero)
    return all_layers[d]

epochs = 1000

for i in range(epochs):
    #print("Epoch: {}".format(i))
    #print("Start weights: {}".format(weights))
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.1,
                        binarizatio_fn=binarization_fn, loss_gradient=loss_gradient)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), binarization_fn(model(input)).numpy()))


Initial weights: [tensor([[-0.6214,  0.7941],
        [ 1.2582,  0.1103]], requires_grad=True), tensor([[ 0.3172, -0.4143],
        [ 1.1341, -0.8095]], requires_grad=True), tensor([[ 1.3722, -0.1198],
        [ 0.3084, -0.1085]], requires_grad=True), tensor([[-0.4801,  0.6995],
        [ 0.1287, -0.5095]], requires_grad=True), tensor([[ 1.3127, -0.3228],
        [-0.3582, -0.6324]], requires_grad=True)]
New weights: [tensor([[-0.6214,  0.7941],
        [ 1.2582,  0.1103]], requires_grad=True), tensor([[ 0.3172, -0.4143],
        [ 1.1341, -0.8095]], requires_grad=True), tensor([[ 1.3722, -0.1198],
        [ 0.3084, -0.1085]], requires_grad=True), tensor([[-0.4801,  0.6995],
        [ 0.1287, -0.5095]], requires_grad=True), tensor([[ 1.3127, -0.3228],
        [-0.3582, -0.6324]], requires_grad=True)]
[[1. 0.]] -> [[0. 0.]]
[[0. 1.]] -> [[0. 0.]]


In [68]:
# with model class
import collections

inputs = [torch.tensor([[1., 0.], [0., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1., 0.], [0., 1.]])]

d=5

layers = [('layer_{}'.format(i+1), LinearBN(2,2, heb_lr=0.01)) for i in range(d)]

model = torch.nn.Sequential(collections.OrderedDict(layers))

print("Layers : {}".format(list(model.named_children())))

print("Initial weights: {}".format([l.weights for _, l in list(model.named_children())]))

for i in range(10000):
    done = learn_model(inputs, desired_outputs, model, learning_rate=0.1)
    if done=="Done":
        print(i)
        break

print("New weights: {}".format([l.weights for _, l in list(model.named_children())]))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))


Layers : [('layer_1', LinearBN()), ('layer_2', LinearBN()), ('layer_3', LinearBN()), ('layer_4', LinearBN()), ('layer_5', LinearBN())]
Initial weights: [Parameter containing:
tensor([[1.2142, 0.9497],
        [0.7405, 1.2641]], requires_grad=True), Parameter containing:
tensor([[1.0411, 1.3990],
        [1.9371, 0.8251]], requires_grad=True), Parameter containing:
tensor([[1.8577, 2.1757],
        [1.1982, 1.9318]], requires_grad=True), Parameter containing:
tensor([[1.5397, 1.9715],
        [1.5662, 2.1869]], requires_grad=True), Parameter containing:
tensor([[1.8676, 1.8878],
        [0.8407, 1.4628]], requires_grad=True)]
331
New weights: [Parameter containing:
tensor([[1.0164, 0.9826],
        [1.0395, 1.0442]], requires_grad=True), Parameter containing:
tensor([[0.4762, 1.0274],
        [0.0588, 0.9711]], requires_grad=True), Parameter containing:
tensor([[0.9757, 0.0706],
        [1.0688, 0.0809]], requires_grad=True), Parameter containing:
tensor([[0.4332, 0.5970],
        [0.06

#### Desingularization via unsupervised learning

Architecture: `input_0, input_1  -> output_0, output_1` 

Desired behavior: no two inputs map to the same outputs

Initial weights: random

In [69]:
inputs = [torch.tensor([[1., 0.], [0., 1.], [1., 1.]], requires_grad=True)]

##### Oja's rule

Oja's rule fails here frequently (this is expected). 

For example, it produces singular outputs of the form `[[0.5005, 1.0010], [0.5005, 1.0010]]`. 

In [70]:
layers = [("layer_1", LinearBN(2,2, oja_lr=0.1))]
model = torch.nn.Sequential(collections.OrderedDict(layers))

print("Initial weights: {}".format([l.weights for _, l in list(model.named_children())]))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

limit = 2000

for i in range(limit):
    done = learn_unsupervised(inputs, model, learning_rate=0.1)

print("New weights: {}".format([l.weights for _, l in list(model.named_children())]))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Initial weights: [Parameter containing:
tensor([[1.5647, 1.0591],
        [0.9207, 2.4553]], requires_grad=True)]
[[1. 0.]
 [0. 1.]
 [1. 1.]] -> [[1. 0.]
 [1. 1.]
 [1. 1.]]
New weights: [Parameter containing:
tensor([[1.0010, 0.5005],
        [0.5005, 1.0010]], requires_grad=True)]
[[1. 0.]
 [0. 1.]
 [1. 1.]] -> [[1. 0.]
 [0. 1.]
 [1. 1.]]


##### Generalized Hebbian rule

It does a better job than Oja's rule. However, it cannot handle a row of weights that are close to zero, because the corresponding neuron will never activate. This must be handled during initialization.  

In [71]:
inputs = [torch.tensor([[1., 0.], [0., 1.]], requires_grad=True)]

layers = [("layer_1", LinearBN(2,2, heb_lr=0.1))]
model = torch.nn.Sequential(collections.OrderedDict(layers))

print("Initial weights: {}".format([l.weights for _, l in list(model.named_children())]))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

limit = 100

for i in range(limit):
    done = learn_unsupervised(inputs, model, learning_rate=0.1)

print("New weights: {}".format([l.weights for _, l in list(model.named_children())]))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Initial weights: [Parameter containing:
tensor([[1.1250, 1.9279],
        [0.9901, 1.8513]], requires_grad=True)]
[[1. 0.]
 [0. 1.]] -> [[1. 0.]
 [1. 1.]]
New weights: [Parameter containing:
tensor([[0.3836, 0.9973],
        [0.1456, 1.2074]], requires_grad=True)]
[[1. 0.]
 [0. 1.]] -> [[0. 0.]
 [0. 1.]]


In [72]:
# try unsupervised learning first, then supervised

inputs = [torch.tensor([[1., 0.], [0., 1.]], requires_grad=True)]
desired_outputs = [torch.tensor([[1., 0.], [0., 1.]])]

d=6
layers = [('layer_{}'.format(i+1), LinearBN(2,2, heb_lr=0.01)) for i in range(d)]

model = torch.nn.Sequential(collections.OrderedDict(layers))

print("Layers : {}".format(list(model.named_children())))

print("Initial weights: {}".format([l.weights for _, l in list(model.named_children())]))

# first, train unsupervised
for i in range(100):
    done = learn_unsupervised(inputs, model, learning_rate=1.)

print("After unsupervised training: {}".format([l.weights for _, l in list(model.named_children())]))

#set heb_lr to zero
for _, l in model.named_children():
    l.heb_lr = torch.tensor([0.02])

# then train supervised
limit = 10000

for i in range(limit):
    done = learn_model(inputs, desired_outputs, model, learning_rate=0.1)
    if done=="Done":
        print("Done after {} episodes".format(i))
        break

print("New weights: {}".format([l.weights for _, l in list(model.named_children())]))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))


Layers : [('layer_1', LinearBN()), ('layer_2', LinearBN()), ('layer_3', LinearBN()), ('layer_4', LinearBN()), ('layer_5', LinearBN()), ('layer_6', LinearBN())]
Initial weights: [Parameter containing:
tensor([[1.5713, 1.3988],
        [0.7556, 1.5309]], requires_grad=True), Parameter containing:
tensor([[0.5151, 0.7059],
        [1.2894, 2.4381]], requires_grad=True), Parameter containing:
tensor([[2.2467, 0.9456],
        [1.6735, 2.4736]], requires_grad=True), Parameter containing:
tensor([[0.7823, 2.0976],
        [0.5894, 2.3530]], requires_grad=True), Parameter containing:
tensor([[0.5794, 1.6384],
        [2.2669, 1.4521]], requires_grad=True), Parameter containing:
tensor([[0.8123, 0.8358],
        [1.8927, 0.9500]], requires_grad=True)]
After unsupervised training: [Parameter containing:
tensor([[1.3836, 0.4318],
        [0.1756, 1.4255]], requires_grad=True), Parameter containing:
tensor([[0.4758, 0.5225],
        [0.6484, 1.3392]], requires_grad=True), Parameter containing:
te